
# Problem Statement :
## Fake News Classification with The Help Of Natural Language Processing Technique.
Fake news detection is a hot topic in the field of natural language processing. We consume news through several mediums throughout the day in our daily routine, but sometimes it becomes difficult to decide which one is fake and which one is authentic. Our job is to create a model which predicts whether a given news is real or fake.

## Dataset Description

id: unique id for a news article

title: the title of a news article

author: author of the news article

text: the text of the article; could be incomplete

label: a label that marks the article as potentially unreliable

  1: unreliable
  
  0: reliable
 
 ### Data Collection
- Dataset Source - https://zenodo.org/record/4561253/files/WELFake_Dataset.csv?download=1
- The data consists of 5 column and 20800 rows.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split,StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from tqdm import tqdm # printing the status bar
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")
pd.pandas.set_option("display.max_columns", None)

### 1. Reading Data

In [ ]:
df = pd.read_csv("data/News_dataset.csv")

print("Shape: ", df.shape)
df.head()


### 2. Data Checks to perform

- Check Missing values
- Check Duplicates
- Check data type
- Check the number of unique values of each column
- Check statistics of data set
- Check various categories present in the different categorical column

### 2.1 Check Missing values

In [ ]:
df.info()

In [ ]:
#df.isna().sum()
df.isnull().sum()

<b>Observation:</b>  There are missing values in the given dataset
- 558 titles features are missing
- 1957 author details are missing
- 39 text features are missing 

### 2.2 Check duplicate values

In [ ]:
df.duplicated().sum()

In [ ]:
df.columns

In [ ]:
print("Duplicate values Features wise:")
#print("Title: ", df["title"].duplicated().sum())
#print("Author: ", df["author"].duplicated().sum())
print("text: ", df["text"].duplicated().sum())

In [ ]:
df[df["text"].duplicated()]

In [ ]:
## check the percentage of null values present in each feature
feature_na =[feature for feature in df.columns if df[feature].isnull().sum() > 0]

for feature in feature_na:
    print(feature, np.round(df[feature].isnull().mean(),4)*100, " % missing values")

In [ ]:
df_text_na = df[df["text"].isnull()]
# Deleteting the rows where text features is null
df.drop(df[df["text"].isnull()].index, inplace = True, axis=0)
df.drop(df[df["id"].isnull()].index, inplace = True, axis=0)
#Update the other features null values with "missing"
df["title"].fillna("missing", inplace=True)
df["author"].fillna("missing", inplace=True)
df.reset_index(inplace=True)
#df.head()

In [ ]:
df.info()

In [ ]:
#Sorting data according to text in ascending order
sorted_data=df.sort_values('text', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [ ]:
sorted_data[sorted_data["text"].duplicated()]

In [ ]:
#How many positive and negative reviews are present in our dataset?
df["label"].value_counts()

# 3.  Text Preprocessing.

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
def preprocess(corpus):
    preprocessed = []
    for sentance in tqdm(corpus):
        #sentance = re.sub(r"http+", "", sentance)
        sentance = re.sub(r"http\S+", "", sentance)
        sentance = BeautifulSoup(sentance, 'lxml').get_text()
        sentance = decontracted(sentance)
        sentance = re.sub("\S*\d\S*", "", sentance).strip()
        sentance = re.sub('[^A-Za-z]+', ' ', sentance)    
        sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
        preprocessed.append(sentance.strip())

    return preprocessed

In [ ]:
import nltk
lm = WordNetLemmatizer()
corpus = []
for i in range (len(df)):
    review = re.sub('^a-zA-Z0-9',' ', df['title'][i])
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(x) for x in review if x not in stopwords]
    review = " ".join(review)
    corpus.append(review)

In [ ]:
corpus[0]


In [ ]:

preprocessed_text = preprocess(df['text'].values)
preprocessed_title = preprocess(df['title'].values)
#preprocessed_author = preprocess(df['author'].values)

#print(preprocessed_text[1500])
#print(preprocessed_title[1500])

In [ ]:
df["text"] = preprocessed_text
#df["title"] = preprocessed_title
#df["author"] = preprocessed_author

df.head()

## 4.  Vectorization (Convert Text data into the Vector)

In [ ]:
tf = TfidfVectorizer()
#X = tf.fit_transform(preprocessed_text).toarray()
X = tf.fit_transform(preprocessed_text).toarray()
X

In [ ]:
X = X.astype(np.uint8)

In [ ]:
df["label"] = df["label"].astype(np.uint8)

y = df["label"]
y.head()

## Data splitting into the train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 10, stratify = y )

In [ ]:
print("X_Train Shape:", X_train.shape)
print("X_test Shape:", X_test.shape)
print("y_train Shape:", y_train.shape)
print("y_test Shape:", y_test.shape)

## Model Building and Evaluation

In [ ]:
model = list()
resample = list()
precision = list()
recall = list()
F1score = list()
AUCROC = list()

def test_eval(clf_model, X_test, y_test, algo=None, sampling=None):
    # Test set prediction
    y_prob=clf_model.predict_proba(X_test)
    y_pred=clf_model.predict(X_test)

    print('Confusion Matrix')
    print('*'*60)
    print(confusion_matrix(y_test,y_pred),"\n")
    print('Classification Report')
    print('*'*60)
    print(classification_report(y_test,y_pred),"\n")
   
          
    model.append(algo)
    precision.append(precision_score(y_test,y_pred))
    recall.append(recall_score(y_test,y_pred))
    F1score.append(f1_score(y_test,y_pred))
    AUCROC.append(roc_auc_score(y_test, y_prob[:,1]))
    resample.append(sampling)

### 1. Logistic Regression

In [ ]:
log_model=LogisticRegression()

params={'C':np.logspace(-10, 1, 15),'class_weight':[None,'balanced'],'penalty':['l1','l2']}

cv = StratifiedKFold(n_splits=5, random_state=100, shuffle=True)

# Create grid search using 5-fold cross validation
clf_LR = GridSearchCV(log_model, params, cv=cv, scoring='roc_auc', n_jobs=-1)
clf_LR.fit(X_train, y_train)
#clf_LR.best_estimator_

In [ ]:
test_eval(clf_LR, X_test, y_test, 'Logistic Regression', 'actual')

### 2. Decision Tree Classifier

In [ ]:
estimators = [2,10,30,50,100]
# Maximum number of depth in each tree:
max_depth = [i for i in range(5,16,2)]
# Minimum number of samples to consider to split a node:
min_samples_split = [2, 5, 10, 15, 20, 50, 100]
# Minimum number of samples to consider at each leaf node:
min_samples_leaf = [1, 2, 5]

In [ ]:
tree_model = DecisionTreeClassifier()

tree_param_grid = { 
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
}

clf_DT = RandomizedSearchCV(tree_model, tree_param_grid, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=2)
clf_DT.fit(X_train, y_train)
clf_DT.best_estimator_

In [ ]:
test_eval(clf_DT, X_test, y_test, 'Decision Tree', 'actual')

### 2. Random Forest Classifier

In [ ]:
rf_model = RandomForestClassifier()

rf_params={'n_estimators':estimators,
           'max_depth':max_depth,
           'min_samples_split':min_samples_split}

clf_RF = RandomizedSearchCV(rf_model, rf_params, cv=cv, scoring='roc_auc', n_jobs=-1, n_iter=20, verbose=2)
clf_RF.fit(X_train, y_train)
clf_RF.best_estimator_

In [ ]:
test_eval(clf_RF, X_test, y_test, 'Random Forest', 'actual')